In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/search-data/questions.json
/kaggle/input/search-data/sentences.json


In [2]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.6 MB/s eta 0:00:0000:0100:01


In [3]:
import faiss

In [4]:
import json
from transformers import AutoModel,AutoTokenizer

In [5]:
with open("/kaggle/input/search-data/questions.json","r") as file:
    questions=json.load(file)

with open("/kaggle/input/search-data/sentences.json","r") as file:
    sentences=json.load(file)

In [6]:
questions[0]

'How many people have died during Black Death?'

In [8]:
sentences[1]

'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.'

In [9]:
model_checkpoint="distilbert-base-uncased"

In [10]:
Tokenizer1=AutoTokenizer.from_pretrained(model_checkpoint)
Model=AutoModel.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
import torch
d = 768
index = faiss.IndexFlatIP(d)

In [19]:
def encoder_func(text):
    input_ids=Tokenizer1(text,truncation=True)['input_ids']
    #attention_mask=Tokenizer1(text,truncation=True)['attention_mask']
    #Attention mask not required as we are encoding each text individually and hence no padding
    input_tensors=torch.tensor([input_ids])
    #attention_tensors=torch.tensor([attention_mask])
    last_hidden_state=Model(input_tensors).last_hidden_state
    Mean_vector=last_hidden_state.mean(dim=1).squeeze()
    return Mean_vector.detach().numpy()

In [21]:
encoder_func("My name is Anish")

array([ 5.98906279e-02,  8.74885544e-02, -9.75136906e-02, -4.33595479e-01,
       -5.30494042e-02,  1.72790498e-01,  2.27933064e-01,  4.83883321e-01,
       -4.88935299e-02, -3.39072227e-01,  1.40936866e-01, -3.01779449e-01,
        1.43032983e-01,  3.98919553e-01, -2.39296943e-01,  7.37558603e-02,
       -3.31314616e-02,  3.78069669e-01, -2.62275785e-01,  3.48408908e-01,
        2.19193354e-01, -1.76458552e-01,  2.25685276e-02,  3.33988994e-01,
        2.22329691e-01,  4.11765724e-01,  1.49305211e-02,  2.22006720e-02,
       -1.35342374e-01, -2.48364523e-01,  1.74667284e-01,  4.60896753e-02,
       -7.84334317e-02,  1.73153952e-01, -8.72350931e-02, -3.39443862e-01,
       -1.26777068e-01, -1.65977448e-01, -4.31770355e-01,  9.76427272e-02,
       -4.88025516e-01, -3.06554854e-01,  1.67025700e-01, -8.45024586e-02,
       -1.78790942e-01, -5.05619943e-01,  5.39841391e-02,  2.83174694e-01,
       -1.25894576e-01,  6.62936941e-02, -6.14896454e-02, -9.46732610e-02,
       -7.61553226e-03, -

In [22]:
quest_vectors=[encoder_func(a) for a in questions]

In [23]:
len(quest_vectors),quest_vectors[1].shape

(10, (768,))

In [24]:
sent_vectors=[encoder_func(text) for text in sentences]

In [25]:
index.add(np.array(sent_vectors))

In [26]:
np.array(sent_vectors).shape

(11, 768)

Markdown Some basic understanding of FAISS

In [17]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [18]:
test_index = faiss.IndexFlatL2(d)   # build the index
print(test_index.is_trained)
test_index.add(xb)                  # add vectors to the index
print(test_index.ntotal)

True
100000


In [20]:
k = 4                          # we want to see 4 nearest neighbors
D, I = test_index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.175174  7.2076287 7.251163 ]
 [0.        6.323565  6.684582  6.799944 ]
 [0.        5.7964087 6.3917365 7.2815127]
 [0.        7.277905  7.5279875 7.6628447]
 [0.        6.763804  7.295122  7.368814 ]]


In [26]:
%%time
D, I = test_index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
CPU times: user 7.57 s, sys: 9.15 ms, total: 7.58 s
Wall time: 3.93 s


Back to our problem!****

In [46]:
def search_func(question,k):
    question_vector=encoder_func(question).reshape(1,d)
    #print(question_vector.shape)
    Distance,Indexes=index.search(question_vector,k)
    #print(Distance,Indexes)
    retrieved_results=[sentences[i] for i in Indexes[0]]
    #print(retrieved_results)
    return {question: dict(zip(retrieved_results,Distance[0]))}

In [47]:
search_func(questions[0],5)

{'How many people have died during Black Death?': {'As of 2018, approximately 37.9 million people are infected with HIV globally.': 52.613434,
  'A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.': 51.77471,
  'The death toll of Spanish Flu is estimated to have been somewhere between 17 million and 50 million, and possibly as high as 100 million, making it one of the deadliest pandemics in human history.': 51.18796,
  'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.': 50.574482,
  'The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.': 47.01428}}

In [35]:
sentences

['A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
 'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
 'Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'As of 2018, approximately 37.9 million people are infected with HIV globally.',
 'Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.',
 'Classic cholera symptom is large amounts of watery diarrhea that lasts a few days. Vomiting and muscle cramps may also occur. Diarrhea can be so severe that it leads within hours to severe dehydration and electrolyte imbalance.',
 'The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome 